In [2]:
# import public packages
import os
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.patches import Rectangle
import scipy
import mne
import sys

from mne.time_frequency import tfr_morlet
from mne.baseline import rescale
from scipy.signal import spectrogram, hann, butter, filtfilt, hilbert
from scipy import signal, interpolate, stats
from scipy.interpolate import make_interp_spline, BSpline
from io import open
from importlib import reload

# import own functions
from utils import find_folders
import dat_preproc
import fix_annot_onsets
import mat2fif
import baseline_correction
import normalization
import power_spectrum

### Add Directories/ Load Files

In [3]:
reload(find_folders)
onedrive = find_folders.get_onedrive_path()
project_path = find_folders.get_onedrive_path("entrainment")
print(project_path)

C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT


In [ ]:
percept_ID = 'sub045'
fname = 'sub-20220516PStn_ses-2023052311114988_run-BrainSense20230523120600.mat'

raw = mne.io.read_raw_fieldtrip(
    os.path.join(
        project_path,
        'data',
        'raw_data',
        'test',
        fname
    ),
    info = None
)

subID = 'Sub045'

In [62]:
raw = mne.io.read_raw_fif(os.path.join(
    project_path,
        'data',
        'Fifs',
        'test',
        'Sub045_FIF.fif'
    )
)

subID = 'Sub045'

Opening raw data file C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT\data\Fifs\test\Sub045_FIF.fif...
    Reading extended channel information
    Range : 0 ... 145611 =      0.000 ...   582.444 secs
Ready.


C:\Users\mathiopv\AppData\Local\Temp\ipykernel_15596\274979519.py:1: RuntimeWarning: This filename (C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT\data\Fifs\test\Sub045_FIF.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(os.path.join(


In [65]:
reload(dat_preproc)
%matplotlib qt
matplotlib.rc('font', size=10)
x = raw.get_data(reject_by_annotation = 'omit',picks=[0,1])
raw = raw
win_samp = 250
noverlap = 0.5
window = hann(win_samp, sym=False)
f, t, Sxx = dat_preproc.fft_rawviz(raw, x, win_samp, noverlap)

In [77]:
fft_name = str(subID)+'_FFT'
print(fft_name)

Sub045_FFT


In [78]:
#Saving the figures & files
fft_fig = os.path.join(project_path, 'figures','FFts','test/')
fft_file = os.path.join(project_path, 'data','FFTs','test/')

plt.savefig(str(fft_fig)+str(fft_name),dpi = 300)

np.save(str(fft_file)+str(fft_name)+'.npy',Sxx)

In [ ]:
#Saving the raw data as a fif file
raw2 = raw.copy()
Draw = raw2.get_data()
info = raw.info
new_raw = mne.io.RawArray(Draw, info)

fif_name = os.path.join(project_path, 'data','Fifs','test/',str(subID)+'_FIF.fif')
new_raw.save(fif_name)


In [103]:
#Zscore Normalization of raw data & plot of new figure
reload(baseline_correction)
data = Sxx
t = t
baseline = (None, None)
raw = raw
stim_ch = 1
bs_data = baseline_correction.baseline_corr(data, t, baseline, raw = raw, stim_ch = 4)

Applying baseline correction (mode: zscore)


In [81]:
fft_fig = os.path.join(project_path, 'figures','FFts','test/')
fft_file = os.path.join(project_path, 'data','FFTs','test/')

plt.savefig(str(fft_fig)+str(fft_name)+'_zscored',dpi = 300)
np.save(str(fft_file)+str(fft_name)+'_zscored'+'.npy',bs_data)

In [82]:
d = {'onset': [52, 162, 550],
    'duration': [30, 15, 10],
    'description': ['StimOff','StimOn','Rebound']
}

epoch_df = pd.DataFrame(data = d)
print(epoch_df)

   onset  duration description
0     52        30     StimOff
1    162        15      StimOn
2    550        10     Rebound


In [83]:
## Print Power Spectra with Raw Data
reload(dat_preproc)
filt_dat = raw.get_data()
time_onsets = epoch_df
window = 250
noverlap = 0.5
ylim2 = 0.4

fig = plt.figure(figsize = (12,5))

plt.subplot(1,2,1)
title1 = str(subID)+ '_LSTN'
ps1 = dat_preproc.epoch_ps(filt_dat, time_onsets, window, noverlap, 0, 0.4, title1)

plt.subplot(1,2,2)
title2 = str(subID)+ '_RSTN'
ps2 = dat_preproc.epoch_ps(filt_dat, time_onsets, window, noverlap, 1, 0.3, title2)

plt.show()

In [84]:
## Printing the Power Spectra (RAW)
ps_all = np.concatenate((ps1,ps2), axis = 0)
ps_df = pd.DataFrame(np.transpose(ps_all),
    columns = ['StimOff_LSTN','StimOn_LSTN','Rebound_LSTN',
    'StimOff_RSTN','StimOn_RSTN','Rebound_RSTN'],
    )
print(ps_df)

     StimOff_LSTN  StimOn_LSTN  Rebound_LSTN  StimOff_RSTN  StimOn_RSTN  \
0        0.629595     2.250421      0.593368      0.943254     4.483053   
1        4.655991     7.438675      2.306109      5.575431    20.407228   
2        5.385942     7.191910      2.116370      5.789026    18.794939   
3        3.905233     4.063774      1.324942      5.085402     6.246797   
4        2.922249     1.870844      1.342228      3.073527     1.780652   
..            ...          ...           ...           ...          ...   
121      0.000331     0.097465      0.000313      0.000364     0.000328   
122      0.000238     0.157975      0.000256      0.000401     0.000518   
123      0.000172     0.594498      0.000188      0.000377     0.000512   
124      0.001480    14.989833      0.000215      0.032225     0.004743   
125      0.002863    27.422658      0.000088      0.062529     0.008729   

     Rebound_RSTN  
0        0.363821  
1        2.210058  
2        3.600086  
3        3.199133  

In [85]:
#Saving the figures & files
fft_fig = os.path.join(project_path, 'figures','Power_Spectra','test/')
fft_file = os.path.join(project_path, 'data','power_spectra','test/')

plt.savefig(str(fft_fig)+str(fft_name)+'_PS',dpi = 150)
ps_df.to_csv(str(fft_file)+str(fft_name)+'_PS.csv')

In [86]:
## Print Power Spectra with Zscored Data
fig, axes = plt.subplots(1,1, figsize = (8,6))

side = 0

ps_m1s0 = np.mean(bs_data[side,:,epoch_df.onset[0]:epoch_df.onset[0]+ epoch_df.duration[0]],1)
ps_m1s1 = np.mean(bs_data[side,:,epoch_df.onset[1]:epoch_df.onset[1]+ epoch_df.duration[1]],1)
ps_reb = np.mean(bs_data[side,:,epoch_df.onset[2]:epoch_df.onset[2]+ epoch_df.duration[2]],1)

plt.plot(np.arange(1,127),ps_m1s0,label = epoch_df.description[0])
plt.plot(np.arange(1,127),ps_m1s1,label = epoch_df.description[1])
plt.plot(np.arange(1,127),ps_reb,label = epoch_df.description[2])
    
plt.xlim(60,90)
plt.xlabel('Frequency [Hz]')
plt.ylabel('LFP Power (z-scored)')
    
plt.legend()
plt.show()


In [87]:
ps_allzscored_df = pd.DataFrame(
    {
        'Spontan': ps_m1s0,
        'StimOn': ps_m1s1,
        'Rebound': ps_reb
    }
)
print(ps_allzscored_df)

      Spontan    StimOn   Rebound
0   -0.303963  0.593510 -0.324023
1   -0.140924  0.209092 -0.436501
2   -0.032595  0.181824 -0.420786
3    0.022664  0.049055 -0.406858
4    0.146444 -0.123558 -0.259307
..        ...       ...       ...
121 -0.073866  1.417390 -0.074140
122 -0.077882  1.356324 -0.077716
123 -0.072251  1.417698 -0.072212
124 -0.891864  4.426541 -0.892313
125 -0.905286  4.195308 -0.905802

[126 rows x 3 columns]


In [88]:
plt.savefig(str(fft_fig)+str(fft_name)+'_Contra_zscoredPS',dpi = 150)
ps_allzscored_df.to_csv(str(fft_file)+str(fft_name)+'_Contra_zscoredPS.csv')

### Checking Spectral Width

In [89]:
#Import the power spectrum csv
ps_tbl = pd.read_csv(os.path.join(
    project_path,
        'data',
        'power_spectra',
        'test',
        'Sub045_FFT_PS.csv'
))

subID = 'Sub045'

x1 = ps_tbl.StimOff_LSTN[50:101]
x2 = ps_tbl.StimOn_LSTN[50:101]
plt.plot(np.arange(0,51),x1)
plt.plot(np.arange(0,51),x2)
plt.xticks(np.arange(0,51,10), labels = np.arange(50,101,10))

plt.ylim(0,0.4)

(0.0, 0.4)

In [90]:
reload(power_spectrum)
%matplotlib qt

height = 0.04

peaks1, results_half1, peaks2, results_half2 = power_spectrum.powerSpectrum_width(x1, x2, height, subID)
print(results_half1)
print(results_half2)

[52 59 82]
[62]
[1.67362357 1.64913827 2.49066218]
[2.01500762]


In [91]:
plt.savefig(os.path.join(
    project_path,
    'figures',
    'Power_Spectra',
    'test',
    str(subID)+'_PSWidth'
),
dpi = 150)

### Average all Power Spectra and Plot them

In [ ]:
## Interpolate and plot power spectrum
trial_ps = pd.read_csv(os.path.join(
    project_path,
        'data',
        'power_spectra',
        'test',
        'Sub005_FFT_PS.csv'
)
)

print(trial_ps)

In [ ]:
reload(dat_preproc)
dat_preproc.mypower(trial_ps['StimOff_RSTN'])
dat_preproc.mypower(trial_ps['StimOn_RSTN'])
plt.xlim(50,100)

In [ ]:
os.path.join(
   project_path,
   'data', 'power_spectra', 'test'
)

In [8]:
print(filename)

Sub005_FFT_Contra_zscoredPS.csv


In [54]:
#loop through all csv files in ZSCORED
from scipy import stats
all_spontan = []
all_subh = []

ps_path = os.path.join(
   project_path,
   'data', 'power_spectra', 'test'
)

'''
for filename in os.listdir(ps_path):
   if filename.endswith('FFT_Ipsi_zscoredPS.csv'):
   #with open(os.path.join(ps_path, filename), 'r') as f: # open in readonly mode
      this_df = pd.read_csv(os.path.join(ps_path,filename))
      this_spontan = this_df['Spontan']
      this_subh = this_df['StimOn']


      all_spontan.append(this_spontan)
      all_subh.append(this_subh)

'''

for filename in os.listdir(ps_path):
   if filename.endswith('FFT_PS.csv'):
      if filename.startswith('Sub006') or filename.startswith('Sub0045'):
   #with open(os.path.join(ps_path, filename), 'r') as f: # open in readonly mode
         this_df = pd.read_csv(os.path.join(ps_path,filename))
         this_spontan = this_df['StimOff_LSTN']
         this_subh = this_df['StimOn_LSTN']
      else: 
         this_df = pd.read_csv(os.path.join(ps_path,filename))
         this_spontan = this_df['StimOff_RSTN']
         this_subh = this_df['StimOn_RSTN']

   all_spontan.append(this_spontan)
   all_subh.append(this_subh)


In [55]:
all_spontan_df = pd.DataFrame(np.transpose(all_spontan))
all_subh_df = pd.DataFrame(np.transpose(all_subh))

In [48]:
reload(dat_preproc)
dat_preproc.mypower(all_spontan_df)
plt.xlim(50,100)

(50.0, 100.0)

In [60]:
%matplotlib qt
plt.plot(all_spontan_df,color = '#1f77b4',lw = 1, alpha = 1, label = 'Med On-Stim Off')
plt.plot(all_subh_df,color = 'red',lw = 1, alpha = 1, label = 'Med On-Stim On')

handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

plt.xlim(50,90)
plt.ylim(0,0.5)
plt.xlabel('Frequency [Hz]')
plt.ylabel('LFP Power')

Text(0, 0.5, 'LFP Power')

In [61]:
plt.savefig(os.path.join(
    project_path,
    'figures',
    'Power_Spectra',
    'test',
    'AllSub_Ipsi_PowerSpectra'
),
dpi = 200)

In [ ]:
all_psSpontan = pd.read_csv(os.path.join(
    project_path,
        'results',
        'AllSpontan_PsZscored.csv'
))


all_psSubharm = pd.read_csv(os.path.join(
    project_path,
        'results',
        'AllSubharm_PsZscored.csv'
))

In [ ]:
all_psSpontan = all_psSpontan.drop('Sub029', axis=1)

In [ ]:
all_ps_mean1 = np.mean(all_psSpontan,1)
all_ps_sem1 = stats.sem(all_psSpontan,1)

all_ps_mean2 = np.mean(all_psSubharm,1)
all_ps_sem2 = stats.sem(all_psSubharm,1)

In [ ]:
plt.plot(np.arange(1,127), all_ps_mean1, label = 'Spontaneous FTG')
plt.fill_between(np.arange(1,127), all_ps_mean1-all_ps_sem1, all_ps_mean1+all_ps_sem1, alpha = 0.2)

plt.plot(np.arange(1,127), all_ps_mean2, label = 'DBS-Induced FTG')
plt.fill_between(np.arange(1,127), all_ps_mean2-all_ps_sem2, all_ps_mean2+all_ps_sem2, alpha = 0.2)

plt.xlim(50, 100)
plt.ylim(-0.4, 2)
plt.legend()

plt.xlabel('Frequency [Hz]')
plt.ylabel('LFP Power [z-scored]')
plt.title('Averaged Power Spectra [N = 6]')

In [ ]:
plt.savefig(os.path.join(
    project_path,
    'results',
    'Avg_PS'
),
dpi = 200)

In [92]:
###Compare Power Spectra Widths

widths_tbl = pd.read_csv(os.path.join(
    project_path,
    'results',
    'PS_Widths.csv'
))
print(widths_tbl)

    SubID  PeakMed  PeakSuh  WidthMed  WidthSubh
0  Sub005       77       65  2.958513   1.613145
1  Sub006       82       65  2.254888   1.256823
2  Sub009       73       65  2.066663   1.257691
3  Sub014       74       65  2.184958   1.324285
4  Sub017       76       63  2.317574   2.179645
5  Sub029       83       71  2.524295   2.007908
6  Sub045       82       62  2.490662   2.015008


In [93]:
import mlxtend

In [94]:
p_value = mlxtend.permutation_test(
    widths_tbl['WidthMed'], widths_tbl['WidthSubh'], paired=True, method="approximate", seed=0, num_rounds=100000
)

print(p_value)

0.015139848601513985


In [102]:
np.round(p_value,decimals = 3)

0.015

In [95]:
dat_to_box = widths_tbl.iloc[:,[3,4]]
plt.boxplot(dat_to_box)


{'whiskers': [<matplotlib.lines.Line2D at 0x1b269ebc160>,
 'caps': [<matplotlib.lines.Line2D at 0x1b269ebce20>,
 'boxes': [<matplotlib.lines.Line2D at 0x1b269ebc7f0>,
 'medians': [<matplotlib.lines.Line2D at 0x1b263f09bb0>,
 'fliers': [<matplotlib.lines.Line2D at 0x1b263f099a0>,
 'means': []}